# --- Day 14: Extended Polymerization --- 

https://adventofcode.com/2021/day/14

## Get Input Data

In [1]:
def parse_data(filename):
    """Read in polymer template and pair insertion data."""
    
    with open(f'../inputs/{filename}') as file:
        lines = [line.strip() for line in file.readlines()]

    polymer_template = lines[0]

    pair_insertion_rules = {}
    for line in lines[2:]:
        # For each key, create a value that contains list of two items.
        # v[0] => The first two characters (this will get used most of the time)
        # v[1] => The full insertion string with all three characters
        # For most insertions, use v[0]; for the last insertion use v[1]
        pair_insertion_rules[line[:2]] = [line[0] + line[-1], line[0] + line[-1] + line[1]] 

    return polymer_template, pair_insertion_rules

In [2]:
test_polymer_template, test_pair_insertion_rules = parse_data('test_polymer_data.txt')
test_polymer_template, test_pair_insertion_rules

('NNCB',
 {'CH': ['CB', 'CBH'],
  'HH': ['HN', 'HNH'],
  'CB': ['CH', 'CHB'],
  'NH': ['NC', 'NCH'],
  'HB': ['HC', 'HCB'],
  'HC': ['HB', 'HBC'],
  'HN': ['HC', 'HCN'],
  'NN': ['NC', 'NCN'],
  'BH': ['BH', 'BHH'],
  'NC': ['NB', 'NBC'],
  'NB': ['NB', 'NBB'],
  'BN': ['BB', 'BBN'],
  'BB': ['BN', 'BNB'],
  'BC': ['BB', 'BBC'],
  'CC': ['CN', 'CNC'],
  'CN': ['CC', 'CCN']})

In [3]:
polymer_template, pair_insertion_rules = parse_data('polymer_data.txt')

## Part 1
---

In [4]:
from more_itertools import pairwise
from collections import Counter

In [5]:
pairwise(test_polymer_template)

<generator object _pairwise at 0x00000132B8980900>

In [6]:
def calc_part_1(template, rules, num_steps):
    """Return the difference of the most and least common counts of a polymer template, 
    after pair insertion rules have been applied for num_steps.
    """

    for _ in range(num_steps):
        pairs = list(pairwise(template))

        next_template = ''
        for i, pair in enumerate(pairs):
            if i != len(pairs) - 1:
                next_template += ''.join(rules[''.join(pair)][0])  # Most insertions just take the first two chars
            else:
                next_template += ''.join(rules[''.join(pair)][1])  # The last insertion takes all three

        template = next_template

    counts = Counter(template)

    most_common = counts.most_common()[0][1]
    least_common = counts.most_common()[-1][1]
    diff = most_common - least_common

    return diff


### Run on Test Data

In [7]:
calc_part_1(test_polymer_template, test_pair_insertion_rules, 10)  # Should return 1588

1588

### Run on Input Data

In [8]:
calc_part_1(polymer_template, pair_insertion_rules, 10)

3118

## Part 2
---

### Run on Test Data

### Run on Input Data